In [4]:
## crawler
import ru_dictionary_parser

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException

import pandas as pd

## database save
import sqlite3

In [16]:
from importlib import reload

# 모듈을 수정한 후에 다시 import
ru_dictionary_parser = reload(ru_dictionary_parser)


## 1. crawl openrussian dictionary

In [2]:
url_list = ["https://en.openrussian.org/list/all"+'/?start='+str(page_num) for page_num in range(50,90050,50)]
url_list.insert(0,"https://en.openrussian.org/list/all")

In [ ]:
## getting word lists
word_links_list = []

## click next page
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

for main_url in url_list:
    driver.get(main_url)

    anchor_tags = driver.find_elements(By.CSS_SELECTOR,'table a')
    word_links = [a.get_attribute('href') for a in anchor_tags]
    word_links_list.extend(word_links)

driver.close()

In [17]:
## get word data
word_data = ru_dictionary_parser.word_parser(word_links_list)

100% |########################################################################|


## 2. make dictionary dataframe

In [18]:
## make data tables

word_title_df = pd.DataFrame(word_data[0], columns = ['canonical_id', 'canonical_form'])
word_pos_df = pd.DataFrame(word_data[1], columns = ['canonical_id', 'pos'])
word_meaning_df = pd.DataFrame(word_data[2], columns = ['canonical_id', 'meaning'])
word_declension_df_ = pd.DataFrame(word_data[3], columns = ['canonical_id', 'tags', 'form'])
word_aspect_df = pd.DataFrame(word_data[4], columns = ['canonical_id', 'verb_aspect'])
word_imperfective_perfective_df = pd.DataFrame(word_data[5], columns = ['canonical_id', 'imperfective_perfective'])

In [36]:
error_links_list = word_data[6].copy()

In [19]:
## word_canonical_df

word_canonical_df = word_title_df\
                    .merge(word_pos_df, how='left',on='canonical_id')\
                    .merge(word_meaning_df, how='left',on='canonical_id')\
                    .merge(word_aspect_df, how='left',on='canonical_id')\
                    .merge(word_imperfective_perfective_df, how='left',on='canonical_id')

word_canonical_df['is_saved'] = 0

word_canonical_df

,canonical_id,canonical_form,pos,meaning,verb_aspect,imperfective_perfective,is_saved
0,0,и,etc,and; also,NaN,NaN,0
1,1,в,etc,in; to,NaN,NaN,0
2,2,не,etc,not,NaN,NaN,0
3,3,на,etc,on (place); on (day/week); on (means of transp...,NaN,NaN,0
4,4,что,etc,what; that,NaN,NaN,0
...,...,...,...,...,...,...,...
87205,87524,прибавля́емый,etc,something which is being done (прибавля́ть),NaN,NaN,0
87206,87525,взыва́ющий,etc,someone who is doing (взыва́ть),NaN,NaN,0
87207,87526,взыва́вший,etc,someone who was doing (взыва́ть),NaN,NaN,0
87208,87527,углуби́вшийся,etc,someone who was doing (углуби́ться),NaN,NaN,0


In [8]:
## word_declension_df

## clean_form column
def strip_stress_marks(text):
    b = text.encode('utf-8')
   # correct error where latin accented ó is used
    b = b.replace(b'\xc3\xb3', b'\xd0\xbe')
   # correct error where latin accented á is used
    b = b.replace(b'\xc3\xa1', b'\xd0\xb0')
   # correct error where latin accented é is used
    b = b.replace(b'\xc3\xa0', b'\xd0\xb5')
   # correct error where latin accented ý is used
    b = b.replace(b'\xc3\xbd', b'\xd1\x83')
   # remove combining diacritical mark
    b = b.replace(b'\xcc\x81',b'').decode()
    return b

In [21]:
## word_declension_df
word_declension_df = word_declension_df_.copy()
word_declension_df['declension_id'] = list(range(word_declension_df.shape[0]))
word_declension_df['clean_form'] = [strip_stress_marks(f) for f in word_declension_df.form]

word_declension_df = word_declension_df[['declension_id', 'form', 'clean_form', 'tags', 'canonical_id']]

In [22]:
word_declension_df

,declension_id,form,clean_form,tags,canonical_id
0,0,тот,тот,masculine_nominative,5
1,1,та,та,feminine_nominative,5
2,2,то,то,neuter_nominative,5
3,3,те,те,nominative_plural,5
4,4,того́,того,genitive_masculine_neuter,5
...,...,...,...,...,...
913478,913478,давящейся,давящейся,feminine_instrumental,87468
913479,913479,давящимися,давящимися,instrumental_plural,87468
913480,913480,давящейся,давящейся,feminine_prepositional,87468
913481,913481,давящемся,давящемся,masculine_neuter_prepositional,87468


## 4. DB save

In [ ]:
########## 격변화 테이블 "-" 데이터 제거

In [41]:
base_cols = ['canonical_form',
             'pos',
             'meaning',
             'verb_aspect',
             'imperfective_perfective',
             'is_saved']

word_canonical_df.drop_duplicates(base_cols, keep='first',ignore_index=True,inplace=True)

In [42]:
word_canonical_df

,canonical_id,canonical_form,pos,meaning,verb_aspect,imperfective_perfective,is_saved
0,0,и,etc,and; also,NaN,NaN,0
1,1,в,etc,in; to,NaN,NaN,0
2,2,не,etc,not,NaN,NaN,0
3,3,на,etc,on (place); on (day/week); on (means of transp...,NaN,NaN,0
4,4,что,etc,what; that,NaN,NaN,0
...,...,...,...,...,...,...,...
86789,87524,прибавля́емый,etc,something which is being done (прибавля́ть),NaN,NaN,0
86790,87525,взыва́ющий,etc,someone who is doing (взыва́ть),NaN,NaN,0
86791,87526,взыва́вший,etc,someone who was doing (взыва́ть),NaN,NaN,0
86792,87527,углуби́вшийся,etc,someone who was doing (углуби́ться),NaN,NaN,0


In [62]:
## save to DB - word_canonical_df

conn = sqlite3.connect('ru_dictionary_temp_db.sqlite3')
cursor = conn.cursor()

## word_canonical_df
#canonical_tuples = [(id_, canonical_form, pos, meaning, aspect_type, aspect_word, is_saved) for id_, canonical_form, pos, meaning, aspect_type, aspect_word, is_saved  in zip(word_canonical_df.canonical_id, word_canonical_df.canonical_form, word_canonical_df.pos, word_canonical_df.meaning, word_canonical_df.verb_aspect, word_canonical_df.imperfective_perfective,word_canonical_df.is_saved)]
#cursor.executemany("INSERT INTO word_canonical VALUES (?, ?, ?, ?, ?, ?, ?)", canonical_tuples)

## word_declension_df
declesion_tuples = [(id_, form, clean_form, tag, canonical_id) for id_, form, clean_form, tag, canonical_id in zip(word_declension_df.declension_id, word_declension_df.form, word_declension_df.clean_form ,word_declension_df.tags, word_declension_df.canonical_id)]
cursor.executemany("INSERT INTO word_declension VALUES (?, ?, ?, ?, ?)", declesion_tuples)

## commit
conn.commit()
conn.close()

### error links re crawl

In [38]:
## word_canonical
conn = sqlite3.connect('ru_dictionary_temp_db.sqlite3')
cursor = conn.cursor()

word_canonical_df = pd.read_sql_query('SELECT * FROM word_canonical', conn)

conn.close()

## word_declension
conn = sqlite3.connect('ru_dictionary_temp_db.sqlite3')
cursor = conn.cursor()

word_declension = pd.read_sql_query('SELECT * FROM word_declension', conn)

conn.close()

word_declension_df = word_declension.merge(word_canonical['canonical_id'], how='inner', on='canonical_id')


In [39]:
## save to DB - word_canonical_df

conn = sqlite3.connect('../rueng_projects/rueng.sqlite3')
cursor = conn.cursor()

## word_canonical_df
canonical_tuples = [(id_, canonical_form, pos, meaning, aspect_type, aspect_word, is_saved) for id_, canonical_form, pos, meaning, aspect_type, aspect_word, is_saved  in zip(word_canonical_df.canonical_id, word_canonical_df.canonical_form, word_canonical_df.pos, word_canonical_df.meaning, word_canonical_df.verb_aspect, word_canonical_df.imperfective_perfective,word_canonical_df.is_saved)]
cursor.executemany("INSERT INTO word_canonical VALUES (?, ?, ?, ?, ?, ?, ?)", canonical_tuples)

## word_declension_df
declesion_tuples = [(id_, form, clean_form, tag, canonical_id) for id_, form, clean_form, tag, canonical_id in zip(word_declension_df.declension_id, word_declension_df.form, word_declension_df.clean_form ,word_declension_df.tags, word_declension_df.canonical_id)]
cursor.executemany("INSERT INTO word_declension VALUES (?, ?, ?, ?, ?)", declesion_tuples)

## commit
conn.commit()
conn.close()

In [3]:
## word_canonical
conn = sqlite3.connect('ru_dictionary_temp_db.sqlite3')
cursor = conn.cursor()

word_canonical_df = pd.read_sql_query('''
                SELECT *
                FROM word_canonical
                WHERE pos = "etc" or pos is null
                
                ''', conn)
## commit
conn.commit()
conn.close()

In [4]:
word_canonical_df

,canonical_id,canonical_form,pos,meaning,verb_aspect,imperfective_perfective,is_saved
0,0,и,etc,and; also,None,None,0
1,1,в,etc,in; to,None,None,0
2,2,не,etc,not,None,None,0
3,3,на,etc,on (place); on (day/week); on (means of transp...,None,None,0
4,4,что,etc,what; that,None,None,0
...,...,...,...,...,...,...,...
35451,87524,прибавля́емый,etc,something which is being done (прибавля́ть),None,None,0
35452,87525,взыва́ющий,etc,someone who is doing (взыва́ть),None,None,0
35453,87526,взыва́вший,etc,someone who was doing (взыва́ть),None,None,0
35454,87527,углуби́вшийся,etc,someone who was doing (углуби́ться),None,None,0


In [12]:
word_declension_df = pd.DataFrame({
    'declension_id': list(range(913483, 913483+word_canonical_df.shape[0])),
    'form': word_canonical_df.canonical_form,
    'clean_form': [strip_stress_marks(w) for w in word_canonical_df.canonical_form],
    'tags': 'main_form',
    'canonical_id': word_canonical_df.canonical_id,
})

In [14]:
## save to DB - word_canonical_df

conn = sqlite3.connect('../rueng_projects/rueng.sqlite3')
cursor = conn.cursor()

## word_declension_df
declesion_tuples = [(id_, form, clean_form, tag, canonical_id) for id_, form, clean_form, tag, canonical_id in zip(word_declension_df.declension_id, word_declension_df.form, word_declension_df.clean_form ,word_declension_df.tags, word_declension_df.canonical_id)]
cursor.executemany("INSERT INTO word_declension VALUES (?, ?, ?, ?, ?)", declesion_tuples)

## commit
conn.commit()
conn.close()

In [5]:
## word_declension
conn = sqlite3.connect('../rueng_projects/rueng.sqlite3')
cursor = conn.cursor()

word_declension = pd.read_sql_query('SELECT * FROM word_declension', conn)

conn.close()

In [9]:
data_list = [(c_form.lower(), id_) for id_, c_form in zip(word_declension.declension_id,word_declension.clean_form)]

In [2]:
'ТАСС'.lower()

'тасс'

In [12]:
## word_declension
conn = sqlite3.connect('../rueng_projects/rueng.sqlite3')
cursor = conn.cursor()

for c_form_lower, id_ in data_list:
    
    cursor.execute('UPDATE word_declension SET clean_form = ? WHERE declension_id = ?;', (c_form_lower, id_))

conn.commit()
conn.close()